## Library & Data

In [1]:
#3.6.8 ver 에서 진행되었습니다.
import sys
print("--sys.version- ")
print(sys.version)

--sys.version- 
3.9.9 (main, Mar  7 2022, 20:41:57) 
[Clang 13.0.0 (clang-1300.0.29.30)]


In [2]:
#!conda install python==3.6

In [3]:
path = '/Users/jaeyeobjeong/Desktop/포트폴리오/프로젝트/온도 예측/data'

In [4]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys, warnings
import seaborn as sns
if not sys.warnoptions: warnings.simplefilter("ignore")
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)
#pd.set_option('max_columns', 1040, 'max_rows', 140)

train = pd.read_csv(f'{path}/train.csv')
test = pd.read_csv(f'{path}/test.csv')

submission = pd.read_csv(f'{path}/sample_submission.csv')

## Data Cleansing & Pre-Processing  

In [5]:
df_month = train[train.loc[:,'Y00':'Y17'].notnull().all(1)].drop(columns = 'Y18')
df_days = train[train['Y18'].notnull()].drop(train.loc[:,'Y00':'Y17'],axis=1)

trainX1, trainX2 = df_month, df_days

trainX1['Y18'] = ((trainX1['Y06'] + trainX1['Y10'] + trainX1['Y12'])/3 + (trainX1['Y00'] + trainX1['Y10'] + trainX1['Y12'])/3)/ 2

trainX1 = trainX1.drop(columns = ['Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08',
       'Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17'])

train = pd.concat([trainX1,trainX2])

train['Y18'] = np.log1p(train['Y18'])

y_train = train['Y18']

data = pd.concat([train.drop(columns=['Y18']),test])

data.index = [i for i in range(len(data))]

In [6]:
# Featrue Rename
features = {
    'temper' : ['X00', 'X07', 'X28', 'X31', 'X32'], 
    'land_pressure' : ['X01', 'X06', 'X22', 'X27', 'X29'], 
    'wind' : ['X02', 'X03', 'X18', 'X24', 'X26'], 
    'rain' : ['X04', 'X10', 'X21', 'X36', 'X39'], 
    'sea_pressure' : ['X05', 'X08', 'X09', 'X23', 'X33'], 
    'sunshine' : ['X11', 'X14', 'X16', 'X19', 'X34'], 
    'humidity' : ['X12', 'X20', 'X30', 'X37', 'X38'], 
    'wind_direction' : ['X13', 'X15', 'X17', 'X25', 'X35']
}

col_names = {}
for feat, v in features.items():
    for i in range(5):
        col_names[f'{v[i]}'] = f'{feat}_{i+1}'


data.rename(columns = col_names,inplace=True)
train.rename(columns = col_names,inplace=True)
test.rename(columns = col_names,inplace=True)

## Feature Engineering

### 기초통계값을 활용한 변수

- 시간을 범주화 하여 해당 시간의 Y18 통계치를 넣음

In [7]:
minute_data = (data.id%144).astype(int)

data['min'] = minute_data

# data['hour'] = pd.Series((data.index%144/6).astype(int))

train['min'] = (train.id%144).astype(int)

def f(x) : 
    if x< 18 : 
        return('t1')
    elif x< 39 : 
        return('t2')
    elif x< 48 : 
        return('t3')
    elif x< 72 : 
        return('t4')
    elif x< 86 : 
        return('t5')
    elif x< 102 : 
        return('t6')
    elif x< 114 : 
        return('t7')
    elif x< 126 : 
        return('t8')
    else : 
        return('t9')

data['time_cat'] = data['min'].apply(f)

train['time_cat'] = train['min'].apply(f)

In [8]:
features = []
f = train.groupby(['time_cat'])['Y18'].agg([('time_cat_sum','sum')]).reset_index()
features.append(f)
f = train.groupby(['time_cat'])['Y18'].agg([('time_cat_mean','mean')]).reset_index()
features.append(f)
f = train.groupby(['time_cat'])['Y18'].agg([('time_cat_std','std')]).reset_index()
features.append(f)

In [9]:
for f in features :
    data = pd.merge(data, f, how='left',on=['time_cat'])
display(data.shape)

(16272, 46)

- 강수량2를 활용하여 0.5를 기준으로 rain 변수를 만든 뒤 두 조건을 고려한 통계치를 넣음

In [10]:
def f2(x) : 
    if x <= 0.5 : 
        return(0)
    else : return(1)

In [11]:
data['rain'] = data['rain_2'].apply(f2)
train['rain'] = train['rain_2'].apply(f2)

In [12]:
features = []
f = train.groupby(['time_cat','rain'])['Y18'].agg([('time_rain_sum','sum')]).reset_index()
features.append(f)
f = train.groupby(['time_cat','rain'])['Y18'].agg([('time_rain_mean','mean')]).reset_index()
features.append(f)
f = train.groupby(['time_cat','rain'])['Y18'].agg([('time_rain_std','std')]).reset_index()
features.append(f)

In [13]:
for f in features :
    data = pd.merge(data, f, how='left',on=['time_cat','rain'])
display(data.shape)

(16272, 50)

In [14]:
data = pd.get_dummies(data=data, columns=['time_cat'])

## 시간에 따른 기온 차를 고려한 변수

- 기온과 관련된 5개 변수에 대해 6시간, 12시간, 24시간, 36시간의 차이에 대한 변수를 만듦

In [15]:
temper_lst = ['temper_1', 'temper_2', 'temper_3', 'temper_4', 'temper_5']

In [16]:
for temper in temper_lst:
    interval = 36
    diff_temper = []
    for i in range(data.shape[0]):
        if i > interval:
            diff = data[temper].iloc[i] - data[temper].iloc[i-interval]
            diff_temper.append(diff)
        else:
            diff = data[temper].iloc[i] - data[temper].iloc[0]
            diff_temper.append(diff)
    data['diff3_{}'.format(temper)] = diff_temper

In [17]:
for temper in temper_lst:
    interval = 72
    diff_temper = []
    for i in range(data.shape[0]):
        if i > interval:
            diff = data[temper].iloc[i] - data[temper].iloc[i-interval]
            diff_temper.append(diff)
        else:
            diff = data[temper].iloc[i] - data[temper].iloc[0]
            diff_temper.append(diff)
    data['diff2_{}'.format(temper)] = diff_temper

In [18]:
for temper in temper_lst:
    interval = 144
    diff_temper = []
    for i in range(data.shape[0]):
        if i > interval:
            diff = data[temper].iloc[i] - data[temper].iloc[i-interval]
            diff_temper.append(diff)
        else:
            diff = data[temper].iloc[i] - data[temper].iloc[0]
            diff_temper.append(diff)
    data['diff_1d{}'.format(temper)] = diff_temper

In [19]:
for temper in temper_lst:
    interval = 216
    diff_temper = []
    for i in range(data.shape[0]):
        if i > interval:
            diff = data[temper].iloc[i] - data[temper].iloc[i-interval]
            diff_temper.append(diff)
        else:
            diff = data[temper].iloc[i] - data[temper].iloc[0]
            diff_temper.append(diff)
    data['diff2_5min_{}'.format(temper)] = diff_temper

## 시간에 따른 습도 차를 고려한 변수

- 습도과 관련된 3개 변수에 대해 6시간, 12시간, 24시간, 36시간의 차이에 대한 변수를 만듦

In [20]:
wet_lst = ['humidity_1', 'humidity_2','humidity_3']

for wet in wet_lst:
    interval = 144
    diff_wet = []
    for i in range(data.shape[0]):
        if i > interval:
            diff = data[wet].iloc[i] - data[wet].iloc[i-interval]
            diff_wet.append(diff)
        else:
            diff = data[wet].iloc[i] - data[wet].iloc[0]
            diff_wet.append(diff)
    data['diff_humidity144_{}'.format(wet)] = diff_wet

In [21]:
for wet in wet_lst:
    interval = 72
    diff_wet = []
    for i in range(data.shape[0]):
        if i > interval:
            diff = data[wet].iloc[i] - data[wet].iloc[i-interval]
            diff_wet.append(diff)
        else:
            diff = data[wet].iloc[i] - data[wet].iloc[0]
            diff_wet.append(diff)
    data['diff_humidity72_{}'.format(wet)] = diff_wet

In [22]:
for wet in wet_lst:
    interval = 216
    diff_wet = []
    for i in range(data.shape[0]):
        if i > interval:
            diff = data[wet].iloc[i] - data[wet].iloc[i-interval]
            diff_wet.append(diff)
        else:
            diff = data[wet].iloc[i] - data[wet].iloc[0]
            diff_wet.append(diff)
    data['diff_humidity216_{}'.format(wet)] = diff_wet

In [23]:
for wet in wet_lst:
    interval = 36
    diff_wet = []
    for i in range(data.shape[0]):
        if i > interval:
            diff = data[wet].iloc[i] - data[wet].iloc[i-interval]
            diff_wet.append(diff)
        else:
            diff = data[wet].iloc[i] - data[wet].iloc[0]
            diff_wet.append(diff)
    data['diff_humidity36_{}'.format(wet)] = diff_wet

In [24]:
#위의 온도 변수와 내용은 동일하지만, 컬럼명이 달라 모델링 시 다른 결과가 나오기 때문에, 추가함.
intervals = [36,72,144,216]

temper_lst = ['temper_1', 'temper_2', 'temper_3', 'temper_4', 'temper_5']
for temper in temper_lst:
    for interval in intervals : 
        diff_temper = []
        for i in range(data.shape[0]):
            if i > interval:
                diff = data[temper].iloc[i] - data[temper].iloc[i-interval]
                diff_temper.append(diff)
            else:
                diff = data[temper].iloc[i] - data[temper].iloc[0]
                diff_temper.append(diff)
        data['diff_{}_{}'.format(interval,temper)] = diff_temper

## 시간에 따른 습도와 일사량 차를 고려한 변수

- 습도과 관련된 5개 변수와 일사량과 관련된 5개의 변수 중 일사량을 측정한 변수 2개에 대해 10분, 1시간의 차이에 대한 변수를 만듦.

In [25]:
#interval단위로 차이를 구하는 함수를 생성. 
def to_per_time(interval, df, colname):
    day = data['min'].nunique() #6*24
    diff = []
    for num_days in range(int(data.shape[0]/data['min'].nunique())):
        for i in range(day):
            if i < interval: #interval단위 이전의 기록들은 차이를 비교할 수 없어, 0으로 대체.
                diff.append(0) 
            else:
                diff.append(df[colname][i] - df[colname][i-interval])
    return diff

In [26]:
intervals = [1,6]
for interval in intervals:
    for i in range(1,6):
        data[f'diff_humidity_{interval}_{i}'] = to_per_time(6,data,f'humidity_{i}')

In [27]:
intervals = [1,6]
for interval in intervals:
    for i in range(1,6):
        try : data[f'diff_sunshine_{interval}_{i}'] = to_per_time(interval,data,f'sunshine_{i}')
        except : pass

### 습도와 일사량의 10분 간 차이에 대한 변수들의 평균 및 곱한 결과

- 10분 전의 일사량과 습도 차이에 대한 변수와 그 두 가지 유형의 변수를 곱한 변수이다.
- 다음의 변수로 일사량과 습도의 단 시간 안에서의 차이를 알 수 있다.
- 다음의 변수는 습도가 높을 경우,일사량이 온도에 미치는 영향을 더 크게 할 수 있다고 판단하여 만듦.

In [28]:
import numpy as np

In [29]:
data['humidity_mean'] = np.mean(data[data.columns[data.columns.str.contains('diff_humidity_1')]],axis=1)
data['sunshine_mean'] = np.mean(data[['diff_sunshine_1_1','diff_sunshine_1_5']],axis=1)

In [30]:
data['sunshine_humidity'] = data['humidity_mean']*data['sunshine_mean']

In [31]:
data.to_csv(f'{path}/for_modeling_data.csv', index = False)